# Brain-Computer Interfaces (Fall 2017, ELE 594)
# Instructor: Yalda Shahriari
# 2nd homework, 10/8/17 (The homework, is due by Oct 18th
# , 11:55 pm).
# Instruction: All the data instructions are the same as homework 1.
# 1-

a) Perform PCA on broadband data using two time windows, one before and one after trial onset
(e.g., - 500 to 0 ms and 100 to 600 ms).

Hint: Separate the requested epochs from the EEG data (e.g. [-500 0ms]). 

Center the data by subtracting the mean of each epoch. 

For each trial obtain the sample covariance matrix (refer to the lecture notes). 

Get the average of the covariances over all the trials. 

Perform eigenvalue decomposition, order the PCs and the eigenvalues in descending order, and proceed to answer the questions.

In [7]:
import scipy.io as spio

sampleEEGdata = spio.loadmat(file_name='../BCI-HW1/sampleEEGdata.mat')

In [9]:
sampleEEGdata.keys()

dict_keys(['__header__', '__globals__', 'EEG', '__version__'])

In [11]:
sampleEEGdata['EEG'].shape

(1, 1)

In [13]:
sampleEEGdata['EEG'][0, 0].dtype

dtype([('setname', 'O'), ('filename', 'O'), ('filepath', 'O'), ('subject', 'O'), ('group', 'O'), ('condition', 'O'), ('session', 'O'), ('comments', 'O'), ('nbchan', 'O'), ('trials', 'O'), ('pnts', 'O'), ('srate', 'O'), ('xmin', 'O'), ('xmax', 'O'), ('times', 'O'), ('data', 'O'), ('icaact', 'O'), ('icawinv', 'O'), ('icasphere', 'O'), ('icaweights', 'O'), ('icachansind', 'O'), ('chanlocs', 'O'), ('urchanlocs', 'O'), ('chaninfo', 'O'), ('ref', 'O'), ('event', 'O'), ('urevent', 'O'), ('eventdescription', 'O'), ('epoch', 'O'), ('epochdescription', 'O'), ('reject', 'O'), ('stats', 'O'), ('specdata', 'O'), ('specicaact', 'O'), ('splinefile', 'O'), ('icasplinefile', 'O'), ('dipfit', 'O'), ('history', 'O'), ('saved', 'O'), ('etc', 'O'), ('spedata', 'O')])

In [14]:
sampleEEGdata['EEG'][0, 0]['data'].shape

(64, 640, 99)

In [16]:
data = sampleEEGdata['EEG'][0, 0]['data']
srate = sampleEEGdata['EEG'][0, 0]['srate']

b) Plot topographical maps and time courses of the first four components. 

To construct the PCA time courses, multiply the PCA weights defined by the pre- and post-trial time windows with the electrode time courses from the entire trial. 

Do you notice any differences in the topographical maps or time courses from before versus after stimulus onset? 

How would you interpret differences and/or similarities?

Note: Make the colormaps in ‘jet’ format and keep the color limit for the topoplots for both
conditions (i.e. pre- and post-stimulus) the same